# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [39]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [40]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,19.2345,-34.4187,283.60,60,100,2.03,ZA,2023-04-21
1,1,yellowknife,-114.3525,62.4560,275.44,64,75,3.60,CA,2023-04-21
2,2,vao,167.4833,-22.6667,298.60,78,19,2.99,NC,2023-04-21
3,3,mogadishu,45.3438,2.0371,302.16,75,100,7.95,SO,2023-04-21
4,4,kodiak,-152.4072,57.7900,280.88,49,0,2.06,US,2023-04-21


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x = "Lat",
    y = "Lng",
    hover_cols = "City",
    geo = True,
    tiles = "OSM",
    color="City",
    legend="right",
    alpha = 0.5
)

# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
clear_sky_warm_df = city_data_df[(pd.to_numeric(city_data_df["Cloudiness"])) == 0 & (pd.to_numeric(city_data_df["Max Temp"]) > 270)]

# Drop any rows with null values
clear_sky_warm_df.dropna(how = "any")

# Display sample data
clear_sky_warm_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,kodiak,-152.4072,57.7900,280.88,49,0,2.06,US,2023-04-21
5,5,jamestown,-79.2353,42.0970,298.95,29,0,9.77,US,2023-04-21
15,15,tiznit,-9.5000,29.5833,287.97,63,0,4.10,MA,2023-04-21
36,36,isa khel,68.8443,36.6702,289.39,40,0,0.98,AF,2023-04-21
42,42,cape town,18.4232,-33.9258,288.92,60,0,1.34,ZA,2023-04-21


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clear_sky_warm_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(1000)

,City,Country,Lat,Lng,Humidity,Hotel Name
4,kodiak,US,-152.4072,57.7900,49,
5,jamestown,US,-79.2353,42.0970,29,
15,tiznit,MA,-9.5000,29.5833,63,
36,isa khel,AF,68.8443,36.6702,40,
42,cape town,ZA,18.4232,-33.9258,60,
...,...,...,...,...,...,...
554,yanam,IN,82.2167,16.7333,92,
561,havelock,US,-76.9013,34.8791,52,
564,madawaska,US,-68.3288,47.3561,43,
576,farah,AF,62.5000,32.5000,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
lat = ""
lon = ""
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #Had to flip the lattude and longitude in the variables as the API was not returning any data. The API documenttion is incorrect so had to flip the inputs.
    latitude = row["Lng"]
    longitude =row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]=f"circle:{longitude},{latitude},{radius}"
    params["bias"]=f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kodiak - nearest hotel: Shelikof Lodge
jamestown - nearest hotel: DoubleTree Jamestown
tiznit - nearest hotel: No hotel found
isa khel - nearest hotel: هوتل چمن شاروالی
cape town - nearest hotel: Townhouse Hotel
port augusta - nearest hotel: Hotel Commonwealth
mount pleasant - nearest hotel: Old Village Post House
tete - nearest hotel: Zambezi Hotel
nome - nearest hotel: No hotel found
gela - nearest hotel: Archeo Hotel
grenfell - nearest hotel: Albion Hotel
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
anamur - nearest hotel: Kemals Pension
san quintin - nearest hotel: Jardines Hotel
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
kusatsu - nearest hotel: スーパーホテル
visby - nearest hotel: Hotell Gute
guerrero negro - nearest hotel: Plaza sal paraiso
baraki barak - nearest hotel: No hotel found
roald - nearest hotel: Vigra fjordhotell
puerto escondido - nearest hotel: Hotel Barlovento
ojinaga - nearest hotel: Hotel
ch

,City,Country,Lat,Lng,Humidity,Hotel Name
4,kodiak,US,-152.4072,57.7900,49,Shelikof Lodge
5,jamestown,US,-79.2353,42.0970,29,DoubleTree Jamestown
15,tiznit,MA,-9.5000,29.5833,63,No hotel found
36,isa khel,AF,68.8443,36.6702,40,هوتل چمن شاروالی
42,cape town,ZA,18.4232,-33.9258,60,Townhouse Hotel
...,...,...,...,...,...,...
554,yanam,IN,82.2167,16.7333,92,No hotel found
561,havelock,US,-76.9013,34.8791,52,Quality Inn Havelock Hwy 70
564,madawaska,US,-68.3288,47.3561,43,Sheraton
576,farah,AF,62.5000,32.5000,12,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot

map_plot = hotel_df.hvplot.points(
    x = "Lat",
    y = "Lng",
    hover_cols = ["City","Hotel Name","Country"],
    geo = True,
    tiles = "OSM",
    color="City",
    legend="right",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Hotel Name,Country)